# Bee Colony Optimization

Cada abeja representa una solución al problema.
Entorno: grafo del problema del viajante timétrico a resolver.
Se representa como una matriz de adyacencia en la que el valor de la posición [i][j] es la distancia entre las ciudades i y j.

In [ ]:
import random
import copy
import json
import time
import tsplib95

### Función cargaCasoDePrueba

Lee un caso de prueba del fichero con nombre _nombreFichero_ y devuelve la matriz de adyacencia correspondiente.

In [ ]:
def cargaCasoDePrueba(nombreFichero):
    with open(nombreFichero, 'r') as archivo:
        matrizAdy = json.loads(archivo.read())
    return matrizAdy

### Función tsplibAMatrizAdyacencia

Convierte el problema dado del formato tsplib a la matriz de adyacencia correspondiente.

In [ ]:
def tsplibAMatrizAdyacencia(problema):
    matrizAdy = [[]] * problema.dimension
    for i in range(problema.dimension):
        matrizAdy[i] = [problema.get_weight(i+1,j+1) for j in range(problema.dimension)]
    return matrizAdy

### Función cargaCasoTsplib

Carga un problema de la tsplib y lo devuelve ya en forma de matriz de adyacencia.

In [ ]:
def cargaCasoTsplib(nombreFichero):
    problema = tsplib95.load(nombreFichero)
    return tsplibAMatrizAdyacencia(problema)

### Función guardarResultados

Dadas las ejecuciones de un algoritmo, las guarda en un fichero formateado adecuadamente para posteriormente cargarlas en el notebook Graficas.ipynb

In [ ]:
def guardarResultados(ejecuciones, problema, algoritmo, num):
    nombreFichero = problema + "_" + algoritmo + "_" + str(num) + ".txt"
    with open(nombreFichero, 'a') as archivo:
        for ejecucion in ejecuciones:
            archivo.write(str(ejecucion[0]) + '\n')
            for abeja in ejecucion[1]:
                archivo.write(str(abeja))
            archivo.write('FIN\n')

## Arista
La clase Arista nos permite representar un camino entre dos ciudades concretas.

In [ ]:
class Arista:
    def __init__(self, desde, hasta, distancia=None):
        self.desde = desde
        self.hasta = hasta
        self.distancia = 1 if distancia is None else distancia

## Entorno
La clase Entorno representa una instancia del problema del viajante a resolver. Contiene una serie determinada de Aristas que crean el grafo a resolver.

In [ ]:
class Entorno:
    def __init__(self, matrizAdy):
        self.conjuntoAristas = set()
        self.aristas = self.creaAristas(matrizAdy)
        self.numVertices = len(matrizAdy)
        self.colmena = 0
        
    def creaAristas(self, matrizAdy):
        aristas = {}
        for i in range(len(matrizAdy)):
            for j in range(i):
                arista = Arista(i, j, matrizAdy[i][j])
                aristas[i,j] = arista
                aristas[j,i] = arista
                self.conjuntoAristas.add(arista)
        return aristas
    
    # Ubica la colmena en una nueva localización aleatoria
    def situaColmena(self):
        self.colmena = random.randrange(self.numVertices)

## Abeja
La clase Abeja representa una solución concreta al problema planteado en el entorno.
Salen de una localización donde se encuentra la colmena y recolectan néctar por el camino (recolectan más néctar cuanto más corto sea el camino recorrido). Al volver a la colmena tienen una cierta probabilidad de bailar para atraer a más abejas por su camino, o bien vuelven a partir ellas solas.

In [ ]:
class Abeja:

    def __init__(self):
        self.nodoActual = None
        self.inicio = None
        self.entorno = None
        self.visitados = []
        self.porVisitar = []
        self.trayecto = []
        self.distancia = 0
        self.seguidor = False
        
    def __str__(self):
        resultado = str(self.distancia) + '\n' + str(self.visitados) + '\n'
        return resultado
    
    def __repr__(self):
        resultado = str(self.distancia) + '\n' + str(self.visitados) + '\n'
        return resultado
        
    # Nos va a interesar poder ordenar las abejas según la distancia de su solución
    # para poder ordenarlas y quedarnos con la mejor.
    def __eq__(self,other):
        return self.distancia == other.distancia
    
    def __lt__(self,other):
        return self.distancia < other.distancia
        
    def inicializar(self, entorno, inicio):
        self.entorno = entorno
        self.inicio = inicio
        self.nodoActual = inicio
        self.visitados = [inicio]
        self.porVisitar = [v for v in range(entorno.numVertices) if v != inicio]
        self.trayecto = []
        self.distancia = 0
        self.seguidor = False
        return self
    
    # Elige un movimiento según BCO y lo efectúa devolviendo la arista elegida.
    def movimiento(self):
        eleccion = self.elegirMovimiento()
        return self.realizaMovimiento(eleccion)
    
    # Elige el movimiento según BCO Devuelve la arista a usar.
    def elegirMovimiento(self):
        # Sólo queda volver al nodo original
        if not self.porVisitar:
            return self.entorno.aristas[self.nodoActual, self.inicio]
        # Sólo queda un nodo por visitar
        if len(self.porVisitar) == 1:
            return self.entorno.aristas[self.nodoActual, self.porVisitar[0]]
        listaAristas = []
        for nodo in self.porVisitar:
            listaAristas.append(self.entorno.aristas[self.nodoActual, nodo])
        listaPesos = [1/arista.distancia for arista in listaAristas]
        pesoAcumulado = sum(listaPesos)
        probabilidades = []
        for peso in listaPesos:
            # Calculamos las probabilidades en %
            probabilidades.append(100 * peso/pesoAcumulado)
        return random.choices(listaAristas, weights=probabilidades)[0]
    
    def realizaMovimiento(self, arista):
        hasta = arista.hasta if self.nodoActual == arista.desde else arista.desde
        if  not (hasta == self.inicio):
            self.visitados.append(hasta)
            self.porVisitar.remove(hasta)
        self.distancia += arista.distancia
        self.nodoActual = hasta
        return arista
    
    def calculaSeguir(self, colonia, probabilidades, posicion):
        if(random.choices(range(len(colonia)), weights=probabilidades)[0] < posicion):
            self.seguidor = True
        else:
            self.seguidor = False
            
    def eligeReclutador(self, listaReclutadores, probabilidades):
        reclutador = random.choices(listaReclutadores, weights=probabilidades)[0]
        self.inicio = copy.deepcopy(reclutador.inicio)
        self.nodoActual = copy.deepcopy(reclutador.nodoActual)
        self.visitados = copy.deepcopy(reclutador.visitados)
        self.porVisitar = copy.deepcopy(reclutador.porVisitar)
        self.trayecto = copy.deepcopy(reclutador.trayecto)
        self.distancia = copy.deepcopy(reclutador.distancia)

## Funciones

### Función crearColonia

Esta función genera una nueva colonia de abejas y las sitúa en la colmena.

In [ ]:
def crearColonia(entorno, numAbejas):
    colonia = []
    for i in range(numAbejas):
            nuevaAbeja = Abeja().inicializar(entorno, 0)
            colonia.append(nuevaAbeja)
    return colonia

In [ ]:
def paso(entorno, colonia):
    for abeja in colonia:
        arista = abeja.movimiento()
        abeja.trayecto.append(arista)

### Funciones BeeSystem

Dado un entorno, un número de iteraciones, y un número de hormigas, aplica la metaheurística del Bee System para intentar obtener una buena solución al problema del TSP. Devuelve la mejor solución encontrada tras todas las iteraciones. En la función BeeSystemConNC se pueden elegir los pasos que se dan antes de volver a la colmena, en BeeSystem se avanza de uno en uno.

In [ ]:
def BeeSystemConNC(entorno, numAbejas, NC):
    tiempoInicio = time.time()
    mejorSolucionGlobal = None
    colonia = crearColonia(entorno, numAbejas)
    numP = 0
    while numP < entorno.numVertices:
        k = 0
        while k < NC and numP < entorno.numVertices: 
            paso(entorno, colonia)
            k += 1
            numP += 1
        colonia = sorted(colonia)
        listaPesos = [1/abeja.distancia for abeja in colonia]
        pesoAcumulado = sum(listaPesos)
        probabilidades = []
        for peso in listaPesos:
            # Calculamos las probabilidades en %
            probabilidades.append(100 * peso/pesoAcumulado)
        i = 0
        for abeja in colonia:
            abeja.calculaSeguir(colonia, probabilidades, i)
            i = i + 1
        listaReclutadores = [abeja for abeja in colonia if (not abeja.seguidor)]
        listaSeguidores = [abeja for abeja in colonia if abeja.seguidor]
        listaPesos = [1/reclutador.distancia for reclutador in listaReclutadores]
        pesoAcumulado = sum(listaPesos)
        probabilidades = []
        for peso in listaPesos:
            # Calculamos las probabilidades en %
            probabilidades.append(100 * peso/pesoAcumulado)
        for seguidor in listaSeguidores:
            seguidor.eligeReclutador(listaReclutadores, probabilidades)    
    mejorSolucionGlobal = sorted(colonia)[0]
    tiempoFin = time.time()
    tiempo = tiempoFin - tiempoInicio
    return colonia, tiempo

In [ ]:
def BeeSystem(entorno, numAbejas):
    tiempoInicio = time.time()
    mejorSolucionGlobal = None
    colonia = crearColonia(entorno, numAbejas)
    for k in range(entorno.numVertices):
        paso(entorno, colonia)
        colonia = sorted(colonia)
        listaPesos = [1/abeja.distancia for abeja in colonia]
        pesoAcumulado = sum(listaPesos)
        probabilidades = []
        for peso in listaPesos:
            # Calculamos las probabilidades en %
            probabilidades.append(100 * peso/pesoAcumulado)
        i = 0
        for abeja in colonia:
            abeja.calculaSeguir(colonia, probabilidades, i)
            i = i + 1
        listaReclutadores = [abeja for abeja in colonia if (not abeja.seguidor)]
        listaSeguidores = [abeja for abeja in colonia if abeja.seguidor]
        listaPesos = [1/reclutador.distancia for reclutador in listaReclutadores]
        pesoAcumulado = sum(listaPesos)
        probabilidades = []
        for peso in listaPesos:
            # Calculamos las probabilidades en %
            probabilidades.append(100 * peso/pesoAcumulado)
        for seguidor in listaSeguidores:
            seguidor.eligeReclutador(listaReclutadores, probabilidades)    
    mejorSolucionGlobal = sorted(colonia)[0]
    tiempoFin = time.time()
    tiempo = tiempoFin - tiempoInicio
    return colonia, tiempo

## Pruebas

In [ ]:
# Cambiar por el nombre del problema deseado
nombreProblema = 'eil51'
matrizAdy = cargaCasoTsplib('ALL_tsp/' + nombreProblema + '.tsp')
# Alterar este range si se quiere ejecutar pruebas con varios números de individuos.
# Por defecto, se empieza por un número de individuos igual al número de ciudades,
# y se va aumentando de 5 en 5 hasta 5 veces esa cifra.
for numAbejas in range(len(matrizAdy), 5 * len(matrizAdy), 5):
    # 20 ejecuciones con cada nº de abejas
    ejecuciones = []
    for i in range(20):
        entorno = Entorno(matrizAdy)
        soluciones, tiempo = BeeSystem(entorno, numAbejas)
        ejecuciones.append((tiempo, soluciones))
    guardarResultados(ejecuciones, nombreProblema, 'BeeSystem', numAbejas)